# Analyzing time traces

<body> The goal of this notebook is to analyze the time traces and probably build a library to analyze future time trace files.
</body>

In [166]:
import sys
import cPickle as pickle
import os
import time

def savePkl(db,fpath):
    ofile = open(fpath,'w')
    pickle.dump(db,ofile)
    ofile.close()
    return True

def loadPkl(pklPath,f):
    fpath = os.path.join(pklPath,f)
    ifile = open(fpath,'r')
    db = pickle.load(ifile)
    ifile.close()
    return db

% matplotlib inline
import numpy as np
import matplotlib.pyplot as plt


In [134]:
import cv2
import numpy as np

def openImage(fname,flag=-1):
    orig = cv2.imread(fname,flag)
    corig = cv2.imread(fname,0)#Always to 8bit color
    cimg = cv2.cvtColor(corig,cv2.COLOR_GRAY2BGR)
    return (orig, corig, cimg)

def saveImage(fname,img):
    cv2.imwrite(fname,img)
    return fname

def simpleShow(img):
    # Shows img when it is a matrix
    plt.imshow(img,cmap='gray')
    plt.show()
    return True

def makeDir(d):
    if not os.path.exists(d): os.makedirs(d)
    return d


In [58]:
# Creating directory and projecting the images from the time trace directory. 
# (a) first image or (b) max intensity projection of the images are computed. 
# This is for the peak finding algorithm to find the peaks.

sourceDir = '/home/jaggu/marcotte_project/boulgakov/microscope/2016-Jan/2016-01-20'
destDir = os.path.join(sourceDir,'projectedImages')
makeDir(destDir)

def projectImages(traceDir, projection='max'):
    allTiff = sorted([os.path.join(traceDir,f) for f in os.listdir(traceDir)])
    final_img = np.zeros((512,512),dtype=np.uint16)
    if projection is 'max':
        for f in allTiff:
            orig, corig, cimg = openImage(f)
            ix = orig > final_img
            final_img[ix] = orig[ix]
    elif projection is 'first':
        orig, corig, cimg = openImage(allTiff[0])
        final_img = orig.copy()
    else:
        raise SystemExit("Input [max, first]")
    return final_img

#imgProjection = 'max'
imgProjection = 'first'
subDir_list = os.walk(sourceDir).next()[1]

for tfname in subDir_list:
    if 'trace' in tfname:
        traceDir = os.path.join(sourceDir,tfname)
        final_img = projectImages(traceDir,projection= imgProjection)
        projectionDir = makeDir(os.path.join(destDir,imgProjection))

        out_fname = os.path.join(projectionDir,os.path.split(traceDir)[1]+'.'+imgProjection+'.tif')
        saveImage(out_fname,final_img)

print "Image projection completed",time.ctime()

Image projection completed Wed Jan 27 16:02:45 2016


In [175]:
"""
##
# PEAK 
         (h_0 rounded to nearest integer (i.e. to nearest pixel),
          w_0 rounded to nearest integer (i.e. to nearest pixel))
# VALUE
         (h_0, w_0, H, A, sigma_h, sigma_w, theta, sub_img, fit_img,
          rmse, r_2, s_n)
          
s_n = (max(sub_img) - mean(img_edge)) / stddev(img_edge)
##

Going to make an output directory in project2. A directory for each traceDir_[imgProjection] is created
The directory structure within each of these traceDir is - 
1. pkl_files
    peak_intensity.pkl;
    peak_bgIntensity.pkl;
    peak_SNR.pkl;
    peak_Status.pkl;
2. converted_images
3. peak_images (randomly chosen)
    [hw]_intensity_trace.png
pbleaching.png; pbleaching.svg
lifetime.png; lifetime.svg
"""
class Peak:
    def __init__(self,pos,subImage):
        self.pos = pos
        self.subImage = subImage
    def intensity(self):
        return np.sum(self.subImage[1:4,1:4])
    def bg_median(self):
        peakIndexList = [(r,c) for r in [1,2,3] for c in [1,2,3]]
        img_edge = [v for rc,v in np.ndenumerate(self.subImage) if not rc in peakIndexList]
        return (np.median(img_edge)*9)
    def bg_mean(self):
        peakIndexList = [(r,c) for r in [1,2,3] for c in [1,2,3]]
        img_edge = [v for rc,v in np.ndenumerate(self.subImage) if not rc in peakIndexList]
        return np.mean(img_edge)*9
    def snr(self):
        peakIndexList = [(r,c) for r in [1,2,3] for c in [1,2,3]]
        img_edge = [v for rc,v in np.ndenumerate(self.subImage) if not rc in peakIndexList]
        snr = ( np.max(self.subImage) - np.mean(img_edge) )/ np.std(img_edge)
        return snr

class PeakTrace:
    """
    The class handles an identified peak (after peak fitting) for the time trace files. 
    The main function is to calculate the subImage for every image in the traceDirectory. It calls the Peak class 
    and gets the statistics involved for the subImage identified. 
    """
    def __init__(self,pos,traceDir):
        self.pos = map(int, pos)
        try: 
            assert type(self.pos[0]) is int and type(self.pos[1]) is int
        except AssertionError:
            print self.pos
            sys.exit(1)
        self.traceFiles = sorted([os.path.join(traceDir,f) for f in os.listdir(traceDir) if f.endswith('.tif')])
    
    def bounds(self,x):
        if x < 0: x = 0
        if x > 511: x = 511
        return x
    
    def getRect(self,p,l=4):
        # 4 is for a 5x5 box
        (h,w) = p
        posRect = [(i,j) for i in range(h-l+2,h+l-1) for j in
                   range(w-l+2,w+l-1)]
        pxLoc = [(self.bounds(i),self.bounds(j)) for (i,j) in posRect]
        return pxLoc
    
    def getSubImage(self,pxL,f,dim=5):
        subimgL = list() 
        orig,gray8bit,cimg = openImage(f)
        for h,w in pxL:
            val = orig[(h,w)]
            subimgL = np.append(subimgL,val)
        sub_image = subimgL.reshape( dim,dim)
        return sub_image
    
    def runThroughFrames(self):
        pxLoc = self.getRect(self.pos)
        intensity_list = list()
        bg_median_list = list()
        snr_list = list()
        status_list = list()
        
        for f in self.traceFiles:
            sub_image = self.getSubImage(pxLoc,f)
            p = Peak(self.pos,sub_image)
            intensity_list.append(p.intensity())
            bg_median_list.append(p.bg_median())
            snr_list.append(p.snr())
            if p.snr()>4: status = True
            else: status = False
            status_list.append(status)
        
        return intensity_list, bg_median_list, snr_list, status_list
            
    
sourceDir = '/home/jaggu/marcotte_project/boulgakov/microscope/2016-Jan/2016-01-20'
peakSourceDir = '/home/jaggu/marcotte_project/boulgakov/microscope/2016-Jan/2016-01-20/projectedImages/first'
outputDir = '/home/jaggu/marcotte_project/current/project2/jaggu/dataAnalysis/microscope1/2016-Jan/2016-01-20/output'
peakID_files = [f for f in os.listdir(peakSourceDir) if f.endswith('pkl')]

peakFile = peakID_files[2]

def analyzeFrames(peakFile):
    # peakFile is the '.pkl' filepath which contains the peak information after image processing. 
    # an output directory structure is created based on the file name
    peak_dict = loadPkl(peakSourceDir,peakFile)

    traceDir_name = peakFile.split('.')[0]
    traceDir = os.path.join(sourceDir,traceDir_name)
    print "Analyzing directory ",traceDir_name,"...",len(peak_dict)

    peakOutput = peakFile.split('.')[0]+'_'+peakFile.split('.')[1]+'_output'
    peak_outputDir = makeDir(os.path.join(outputDir,peakOutput))
    
    pklPath = os.path.join(peak_outputDir,'pklFiles')
    if not os.path.exists(pklPath): makeDir(pklPath)
    
    peak_intensity_dict = dict()
    peak_snr_dict = dict()
    peak_bg_median_dict = dict()
    peak_status_dict = dict()

    for pos,val in peak_dict.items():
        # For each peak pos, it runs through all the frames in the time trace directory
        ptrace = PeakTrace(pos,traceDir)
        intensity_list, bg_median_list, snr_list, status_list = ptrace.runThroughFrames()
        peak_intensity_dict[pos] = intensity_list
        peak_bg_median_dict[pos] = bg_median_list
        peak_snr_dict[pos] = snr_list
        peak_status_dict[pos] = status_list

    savePkl(peak_intensity_dict,os.path.join(pklPath,'peak_intensity.dict.pkl'))
    savePkl(peak_bg_median_dict,os.path.join(pklPath,'peak_bg_median.dict.pkl'))
    savePkl(peak_snr_dict,os.path.join(pklPath,'peak_snr.dict.pkl'))
    savePkl(peak_status_dict,os.path.join(pklPath,'peak_status.dict.pkl'))
    
    return traceDir, pklPath

t0 = time.ctime()
traceDir, pklPath = analyzeFrames(peakFile)
print t0, time.ctime()

Analyzing directory  150120_P025-20nM_MeOH_647_1s_trace001 ... 232
Thu Jan 28 12:39:27 2016 Thu Jan 28 12:47:03 2016


*** 
### General image statistics for the time traces
<body> 
The statistics are - (a) Photobleaching curve (b) Lifetime histogram (i.e frequency of frames with peptides till the first OFF state)
</body>

In [249]:
from __future__ import division
import collections
import numpy


outputDir = '/home/jaggu/marcotte_project/current/project2/jaggu/dataAnalysis/microscope1/2016-Jan/2016-01-20/output'

traceFile = '150120_P025-20nM_MeOH_647_1s_trace001'
trace_outputDir = os.path.join(outputDir,traceFile+'_first_output')
pklPath = os.path.join(trace_outputDir,'pklFiles')

# MATH MODELS

def model_func(t, A, K, C):
    return A * np.exp(K * t) + C

def fit_exp_linear(t, y, C=0):
    y0 = np.array(y,dtype=float)
    y = np.array(y,dtype=float)
    t = np.array(t,dtype=int)
    y = y - C
    y = np.log(y)
    K, A_log = np.polyfit(t, y, 1)
    A = np.exp(A_log)
    y_fit = model_func(t,A,K,C)
    rmse = np.sqrt(((y_fit - y0) ** 2).mean())
    
    return A, K, y_fit, rmse


def computeFrameONs(pklPath):
    peak_status_fname = 'peak_status.dict.pkl'
    peak_status_dict = loadPkl(pklPath,peak_status_fname)
    
    frames_tillOff = list()
    nbrFrames = len(peak_status_dict.values()[0])
    sum_array = np.array([False]*nbrFrames,dtype=bool)
  
    for pos,status_list in peak_status_dict.items():
        frame_firstOff = next(i for i,status in enumerate(status_list) if status is False)
        frames_tillOff.append(frame_firstOff)
        sum_array = np.vstack((sum_array,status_list))
    
    frameCounts = np.sum(sum_array,axis=0)
    peakONCounts = np.sum(sum_array,axis=1)
    return frameCounts,peakONCounts,frames_tillOff

def plotPbleaching(frameCounts,trace_outputDir):
    
    def _plot(y,norm=False):
        t = range(len(y))
        A, K, y_fit, rmse = fit_exp_linear(t,y)
        plt.plot(t,y,marker='o',markerfacecolor='#0072B2',markersize=6)
        plt.plot(t,y_fit,color='#E69F00',linewidth=2)
        plt.xlabel('Frames')
        plt.ylabel('Number of peptides ON')
        
        if norm: fname1 = 'pbleaching.norm'
        else: fname1 = 'pbleaching'
        plt.savefig(os.path.join(trace_outputDir,fname1+'.svg'))
        plt.savefig(os.path.join(trace_outputDir,fname1+'.png'))
        plt.close()
        
    _plot(frameCounts)
    norm_frameCounts = [float(item/max(frameCounts)) for item in frameCounts]
    _plot(norm_frameCounts,norm=True)

    

frameCounts, peakONCounts, frames_tillOff = computeFrameONs(pklPath)
plotPbleaching(frameCounts,trace_outputDir)
print trace_outputDir

/home/jaggu/marcotte_project/current/project2/jaggu/dataAnalysis/microscope1/2016-Jan/2016-01-20/output/150120_P025-20nM_MeOH_647_1s_trace001_first_output


3